In [ ]:
%%capture
# If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/accelerate-0.27.2-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/transformers-4.38.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms
!pip install --no-index /kaggle/input/making-wheels-of-necessary-packages-for-hf-llms/optimum-1.17.1-py3-none-any.whl --find-links=/kaggle/input/making-wheels-of-necessary-packages-for-hf-llms

In [ ]:
import bitsandbytes
import accelerate
import transformers
import optimum

## bitsandbytes:

Ця бібліотека використовується для оптимізації тренування глибоких нейронних мереж, зокрема забезпечуючи більш ефективні варіанти виконання операцій на низькому рівні, таких як робота з вагами моделі у форматі з меншою точністю (наприклад, 8-бітне квантування). Це може покращити швидкість навчання та зменшити вимоги до пам'яті без значної втрати точності.
## accelerate
Розроблений командою Hugging Face, цей пакет допомагає легко переносити код, який працює на одному пристрої, на багато GPU або навіть TPU, без потреби в розробці спеціалізованого коду для паралельного виконання. Він спрощує масштабування тренування моделей на різних обчислювальних платформах.
## transformers
Ще одна бібліотека від Hugging Face, яка забезпечує доступ до великої кількості переднавчених моделей на базі трансформерів (як BERT, GPT), що можна використовувати для різноманітних завдань з обробки природної мови (NLP), таких як класифікація текстів, генерація текстів, аналіз сентименту тощо.
## optimum
Цей пакет також розроблений Hugging Face і призначений для оптимізації моделей, особливо трансформерів, для конкретних обчислювальних платформ. Він включає інструменти для квантування, прунінгу (обрізання), та інших технік оптимізації, щоб зробити моделі швидшими та ефективнішими під час виробництва.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig



MODEL_PATH = "/kaggle/input/gemma/transformers/7b-it/1"
#MODEL_PATH = "/kaggle/input/gemma/transformers/2b-it/2"
#MODEL_PATH = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
#MODEL_PATH = "/kaggle/input/mixtral/pytorch/8x7b-instruct-v0.1-hf/1"
#MODEL_PATH = "/kaggle/input/phi/transformers/2/1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, model_max_length=4096)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config=quantization_config,
)


## Імпорти:

torch: основний пакет для роботи з тензорами і нейронними мережами в Python.
AutoModelForCausalLM: клас з бібліотеки transformers для автоматичного завантаження моделей, що генерують текст (каузальних мовних моделей).
AutoTokenizer: клас для автоматичного завантаження токенізатора, який перетворює текст у формат, придатний для моделі.
BitsAndBytesConfig, AutoConfig: класи для налаштування конфігурації моделі, включаючи параметри квантування.
## Вказання шляху до моделі:
MODEL_PATH: змінна, що містить шлях до папки, де збережено переднавчену модель. Коментовані рядки показують альтернативні шляхи, які можуть бути використані.
## Конфігурація квантування:
quantization_config: створює конфігурацію для квантування моделі до 4-бітової точності, використовуючи специфічні параметри з бібліотеки bitsandbytes. Це зменшує вимоги до пам'яті і може прискорити виконання моделі, зберігаючи при цьому достатню точність.

## Ініціалізація токенізатора:
tokenizer: завантажує токенізатор для моделі з вказаного шляху і налаштовує максимальну довжину вхідних даних (4096 символів). Також налаштовується токенізатор на використання лівого вирівнювання для доповнення токенів.

## Ініціалізація моделі:
model: завантажує модель для каузального мовного моделювання з вказаного шляху, використовуючи автоматичне визначення пристрою (GPU, TPU тощо) та приймаючи конфігурацію квантування. Опція trust_remote_code дозволяє завантажувати та використовувати код зі вказаного джерела, що може бути потрібним для коректної роботи деяких моделей.

In [ ]:
import pandas as pd
from string import Template
from pathlib import Path
import numpy as np
import os

import warnings
warnings.simplefilter("ignore")

import torch
from transformers import pipeline, AutoTokenizer

from tqdm.notebook import tqdm

data_path = Path('/kaggle/input/llm-prompt-recovery')

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    test = pd.read_csv(data_path / 'test.csv', index_col='id')
    test["rewrite_prompt"] = "-"
else:
    test = pd.read_csv(data_path / 'train.csv', index_col='id')
test.head()

Імпорт функції pipeline та класу AutoTokenizer з бібліотеки transformers для використання моделей обробки природної мови.
Імпорт tqdm з модулю tqdm.notebook для відображення інтерактивних прогрес-барів у Jupyter Notebook.

## Зчитування даних:

data_path: змінна, яка визначає шлях до даних, збережених у форматі, зручному для Kaggle.
Перевірка змінної оточення KAGGLE_IS_COMPETITION_RERUN для визначення, чи перезапущено конкурс Kaggle. Це важливо для того, щоб розуміти, які дані використовувати (тестові чи тренувальні).
Залежно від умови, зчитуються відповідні файли CSV (тестові чи тренувальні) з використанням pandas.read_csv із зазначенням індексу.
## Відображення даних:

test.head(): показує перші п'ять рядків датафрейму, який містить тестові або тренувальні дані.

In [ ]:
from torch import nn
class Perplexity(nn.Module):
    def __init__(self, reduce: bool = True):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss()
        self.reduce = reduce

    def forward(self, logits, labels):
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        perplexity = []
        for i in range(labels.shape[0]):
            perplexity.append(self.loss_fn(shift_logits[i], shift_labels[i]))
        perplexity = torch.stack(perplexity, dim=0)
        #perplexity = torch.exp(perplexity)
        if self.reduce:
            perplexity = torch.mean(perplexity)
        return perplexity 
    
perp = Perplexity()

Цей код визначає клас Perplexity, який є підкласом torch.nn.Module із бібліотеки PyTorch. Цей клас призначений для розрахунку перплексії, яка є метрикою для оцінки якості моделей машинного навчання, зокрема, моделей, які генерують послідовності (наприклад, мовні моделі). Ось докладний опис компонентів цього класу:

## __init__
reduce: Цей параметр визначає, чи потрібно зводити масив значень перплексії до одного середнього значення. Якщо True, то перплексія буде середнім значенням по всім прикладам у батчі; якщо False, то перплексія повертається для кожного окремого прикладу.
loss_fn: Використовується функція втрат nn.CrossEntropyLoss, яка є стандартною для класифікації з м'якими мітками (soft labels) та є зручною для обчислення перплексії.
## forward
Приймає два аргументи: logits (логіти виходу моделі) та labels (істинні мітки класів).
shift_logits та shift_labels: Зміщує тензори, щоб відповідні мітки відповідали виходам моделі, зміщеним на один крок в часі. Це зміщення необхідне для правильного обчислення перплексії, де прогноз для кожного кроку часу порівнюється з наступною міткою у послідовності.
Внутрішній цикл ітерує через всі приклади у батчі, обчислюючи перплексію для кожного з них окремо.
torch.stack: Об'єднує масив значень перплексії у тензор.
Коментований рядок torch.exp(perplexity): Зазвичай, для перплексії потрібно взяти експоненту від середнього значення крос-ентропії, щоб перевести її у більш інтуїтивно зрозумілу метрику. Однак, в цьому прикладі цей рядок закоментовано, що може бути помилкою або свідомим вибором для повернення сирової крос-ентропії.
Умовний оператор перевіряє, чи потрібно зменшити розмірність результату до одного середнього значення.

In [ ]:
def format_prompt(row, prompt):
    prompt_ = f"""<start_of_turn>user
{prompt}
{row["original_text"]}<end_of_turn>
<start_of_turn>model
{row["rewritten_text"]}<end_of_turn>"""
    return prompt_

In [ ]:
rewrite_prompts = [
    'Please improve this text using the writing style with maintaining the original meaning but altering the tone.',
]

In [ ]:
preds = []

for idx, row in tqdm(test.iterrows(), total=len(test)):
        
    
    with torch.no_grad():
        perps = []
        samples = []
        for prompt in rewrite_prompts:
            samples.append(format_prompt(row, prompt))
        inputs = tokenizer(samples, return_tensors="pt", add_special_tokens=False, padding=True, truncation=True).to("cuda")

        output = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        output = output.logits
        labels = inputs["input_ids"]
        labels.masked_fill_(~inputs["attention_mask"].bool(), -100)
        for j in range(len(rewrite_prompts)):
            p = perp(output[j].unsqueeze(0), labels[j].unsqueeze(0))
            perps.append(p.detach().cpu())
            
        del inputs
        del labels
        del output
        del p

    perps = np.array(perps)
        
    predictions = [np.array(rewrite_prompts)[np.argsort(perps)][0]]
    preds.append(predictions[0])
    print(preds)

Цей фрагмент коду на Python призначений для оцінювання серії підказок шляхом вимірювання їхньої перплексії за допомогою мовної моделі, а потім вибору підказки з найнижчою перплексією для кожного прикладу в наборі даних. Цей процес зазвичай використовується у сценаріях, коли ви хочете знайти найбільш "природну" або "ймовірну" переробку даного тексту на основі прогнозів моделі.

## Основні змінні та налаштування
preds: Список для зберігання вибраної підказки з найнижчою перплексією для кожного рядка в наборі даних.
test: DataFrame, який, ймовірно, містить текстові дані для обробки.
rewrite_prompts: Список альтернативних підказок для тестування кожного рядка в наборі даних.
Цикл через дані
Ітерація через test:
for idx, row in tqdm(test.iterrows(), total=len(test)): Ітерує через кожен рядок у DataFrame test. Використовує tqdm для прогрес-бару, що вказує, як далеко просунувся процес.
Генерація та оцінка підказок
Для кожного рядка:
## Генерація зразків:

samples = []: Тимчасовий список для зберігання відформатованих підказок.
for prompt in rewrite_prompts: Проходить через кожну попередньо визначену підказку.
samples.append(format_prompt(row, prompt)): Викликає format_prompt, щоб відформатувати кожну підказку з поточними даними рядка і додає її до samples.
## Токенізація та підготовка вхідних даних:

inputs = tokenizer(...): Використовує токенізатор для перетворення текстових зразків у формат, придатний для моделі, включно з налаштуванням на відповідний пристрій (cuda).
## Виведення моделі:

output = model(...): Надсилає токенізовані вхідні дані до моделі, щоб отримати логіти.
labels = inputs["input_ids"]: Встановлює мітки для розрахунку перплексії (зазвичай мітки - зсунуті ID вхідних даних для мовних моделей).
labels.masked_fill_(...): Маскує мітки, де маски уваги є хибними, використовуючи -100 для ігнорування цих у розрахунку втрат.
## Розрахунок перплексії:

perps = []: Список для зберігання розрахованих перплексій для кожної підказки.
for j in range(len(rewrite_prompts)): Ітерує через кожну відповідь від моделі.
p = perp(...): Викликає екземпляр класу Perplexity, щоб обчислити перплексію для кожної підказки.
perps.append(p.detach().cpu()): Відокремлює перплексію від GPU, переміщує на CPU і додає до списку.
Очищення та вибір
Очищення:

del inputs, labels, output, p: Явно видаляє тензори, щоб звільнити пам'ять GPU.
Вибір найкращої підказки:

perps = np.array(perps): Перетворює список перплексій у масив numpy.
predictions = [np.array(rewrite_prompts)[np.argsort(perps)][0]]: Знаходить індекс найменшої перплексії, отримує відповідну підказку та зберігає її.
preds.append(predictions[0]): Додає кращу підказку до кінцевого списку.
print(preds): Опціонально виводить список кращих підказок для відстеження прогресу або для відлагодження.

In [ ]:
submission = pd.read_csv(data_path / 'sample_submission.csv')
submission["rewrite_prompt"] = preds

In [ ]:
submission.to_csv('submission.csv', index=False)